# Tree Methods Consulting Project

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [3]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treecode').getOrCreate()

# Load training data
data = spark.read.csv('/FileStore/tables/dog_food.csv',inferSchema=True,header=True)

data.columns

Out[2]: ['A', 'B', 'C', 'D', 'Spoiled']

In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],
                           outputCol='features')

assembler_data = assembler.transform(data)

final_data = assembler_data.select('features', assembler_data.Spoiled.alias('label'))

final_data.show(5)

+------------------+-----+
 features|label|
+------------------+-----+
[4.0,2.0,12.0,3.0]| 1.0|
[5.0,6.0,12.0,7.0]| 1.0|
[6.0,2.0,13.0,6.0]| 1.0|
[4.0,2.0,12.0,1.0]| 1.0|
[4.0,2.0,12.0,3.0]| 1.0|
+------------------+-----+
only showing top 5 rows

In [5]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbt = GBTClassifier()

dtc_preds = dtc.fit(final_data)
rfc_preds = rfc.fit(final_data)
gbt_preds = gbt.fit(final_data)

In [6]:
data.show(5)

final_data.head(5)

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
+---+---+----+---+-------+
only showing top 5 rows

Out[30]: [Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), label=1.0),
 Row(features=DenseVector([5.0, 6.0, 12.0, 7.0]), label=1.0),
 Row(features=DenseVector([6.0, 2.0, 13.0, 6.0]), label=1.0),
 Row(features=DenseVector([4.0, 2.0, 12.0, 1.0]), label=1.0),
 Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), label=1.0)]

In [7]:
print(dtc_preds.featureImportances.toArray())
print(rfc_preds.featureImportances.toArray())
print(gbt_preds.featureImportances.toArray())


[0. 0.00191078 0.98316765 0.01492157]
[0.0188604 0.0141325 0.93835336 0.02865374]
[0.02962567 0.03830179 0.82862772 0.10344481]

In [8]:
# The preservative C is the component which is spoiling the dog food.

# Good Luck!